# Pré-processamento de dados e validação

Oi pessoal!

Nessa aula, vamos falar um pouco sobre pré-processamento de dados e validação de modelos! Vamos aprender:
- Como processar seus dados para entegá-los da melhor forma para seu modelo.
- A prever quem vive e quem morre em um acidente marítimo.
- Métricas de avaliação.
- Estratégias de validação.
- Como submeter em uma competição do Kaggle.

Nós vamos passar por diversas técnicas e colocá-las em prática em um dos datasets mais clássicos: Sobreviventes do Titanic!

Espero que vocês gostem, não esqueçam de deixar o like e adicionar o notebook aos seus favoritos.

### *IMPORTANTE BAIXAR OS DADOS*

Para baixar os dados você vai precisar se cadastrar no Kaggle - explico no próximo tópico o que é esse site.

Os dados podem ser baixados em: https://www.kaggle.com/c/titanic/data  
Nesse link você também encontra uma descrição sobre cada uma das features do dataset.

Três arquivos serão fornecidos:

- Dados de treino: conjunto de dados dos quais sabemos a resposta.
- Dados de teste: conjunto de dados para o qual faremos nossas previsões.
- Exemplo de submissão: Arquivo que nos mostra o formato de uma submissão para essa competição.

## O que é o Kaggle?

O Kaggle é a maior comunidade de Ciência de Dados do mundo. Nele, você encontra diversos datasets, estudos de pessoas do mundo todo e ainda pode participar de competições organizadas pelo site - muitas delas valendo dinheiro.

Nosso objetivo é que, ao fim dessa aula, você faça sua primeira submissão em uma competição do Kaggle. A competição é a do [Titanic](https://www.kaggle.com/c/titanic/overview), e ela foi criada apenas para fins educacionais.

## Importando os dados

Vamos começar importando os dados que acabamos de baixar e dando uma primeira olhada neles. Nós vamos assumir que vocês ja sabem Pandas mas, qualquer dúvida que surgir, não deixem de dar uma olhada na aula específica do assunto, pesquisar no Google (a documentação dessas bibliotecas é bem feita) ou perguntar pra gente!

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

combine = [train, test]

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tipos de variáveis

Vamos começar entendendo melhor sobre o nosso material de estudo: os dados. Um dataset pode ser visto como um conjunto de *data objects*, normalmente chamados de instâncias, observações, vetores, linhas ou exemplos. Esses são cada uma das linhas do nosso conjunto de dados.

Cada um desses *data objects* é definido por um conjunto de varíaveis - ou *features*. Que expressam, de diferentes maneiras, as características da nossa instância. Ao descrever uma sala de aula, por exemplo, podemos ter as seguintes features: localização, capacidade, quadro negro ou branco, quantidade de tomadas, número de portas, se possui um ar condicionado. A ideia é que essas variáveis sejam capazes de descrever as caracteristicas básicas das salas.

Existem diferentes tipos de variáveis, eles são:

- Numéricas
    - Discretas (representam contagens) e.g. número de cadeiras em uma sala
    - Contínuas (assumem valores em um intervalo contínuo) e.g preço de uma passagem
- Categóricas
    - Ordinais (a ordem importa) e.g. tamanho de roupa (P < M < G)
    - Nominais (não existe ordem entre os valores) e.g. cor da roupa

Vamos identificar agora os tipos de variáveis que temos no dataset do Titanic. Todo *DataFrame* possui um atributo chamado **dtypes**, que nos mostra o tipo de cada variável.

In [4]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Agora nós sabemos de que tipo cada variável é. O tipo *object* indica que a variável é uma string, que trataremos como variáveis categóricas.

É importante perceber que nem toda variável númerica é **realmente** numérica. Pegue a variável *Pclass*, por exemplo, seus valores são: 1a, 2a ou 3a classe. Esses valores já foram processados utilizando uma das técnicas que veremos nesse notebook e, por isso, ela agora é tratada como um número pelo código, embora permaneça sendo uma feature categórica.

Para entender de verdade o tipo de cada variável, o melhor que podemos fazer é ler a documentação que é normalmente fornecida com os dados. Abaixo, separaremos as variáveis nos tipos que mencionamos anteriormente.

**Quais variáveis são numéricas?**  
Contínuas: Age, Fare.<br> Discretas: SibSp, Parch.

**Quais variáveis são categóricas?**  
Nominais: Survived, Sex, and Embarked.<br> Ordinais: Pclass.

## Valores faltantes

Antes de começarmos a criar nossas features - ou fazer qualquer coisa - é bom sempre dar uma olhada no nosso conjunto de dados. Podemos ver se nenhum dado está faltando, verificar se os dados fazem sentido, se estão consistentes e coisas do tipo.

Além disso, é uma boa ideia fazer perguntas e buscar as respostas no seu dataset, assim como criar gráficos que te façam entender melhor seus dados e que forneçam insights sobre eles. Essa etapa do processo é chamada de Análise Exploratória de Dados (EDA).

Como o foco da aula é no pré-processamento, vamos apenas tratar dos valores faltantes pois eles podem nos trazer alguns problemas mais pra frente. Utilizando os comandos mostrados abaixo, você pode descobrir quantos valores estão faltando em cada coluna do seu dataset.

In [5]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Existem diversas formas de preencher os *missing values*, por exemplo:
 - Média ou mediana de toda a coluna.
 - Agrupar os dados por outras features e pegar média ou mediana depois. Por exemplo, digamos que queremos preencher valores vazios de idade. É possível que pessoas de diferentes classes tenham médias de idades diferentes, então, na hora de preencher podemos olhar para a média ou mediana da idade daqueles que estão na mesma classe que nosso exemplo com idade vazia está.
 - Preencher com 0.
 - Excluir as linhas que contém valores nulos.
 
Todas essas formas são válidas e tem seus prós e contras. Por motivo de simplicidade, vamos preencher os valores faltantes em Age e Fare com a mediana da coluna e em Embarked com o valor mais frequente. Não vamos preencher Cabin pois não usaremos essa feature.

In [7]:
embarked_mode = train['Embarked'].mode()[0]
train['Embarked'] = train['Embarked'].fillna(embarked_mode)
test['Embarked'] = test['Embarked'].fillna(embarked_mode)

train_median_fare = train['Fare'].dropna().median()
train['Fare'] = train['Fare'].fillna(train_median_fare)
test['Fare'] = test['Fare'].fillna(train_median_fare)

train_median_age = train['Age'].dropna().median()
train['Age'] = train['Age'].fillna(train_median_age)
test['Age'] = test['Age'].fillna(train_median_age)

## Feature engineering

Protinho, agora podemos começar a criar algumas features básicas.

Uma das partes mais importantes do pré-processamento de dados é a criação de features que ajudem seu modelo a encontrar padrões mais facilmente. Nessa parte do notebook, nós vamos criar algumas novas variáveis e analisar como elas se relacionam com a sobrevivência dos passageiros.

As features criadas aqui foram retiradas de: https://www.kaggle.com/startupsci/titanic-data-science-solutions

### Título do passageiro

Se pensarmos bem, o nome de um indivíduo não tem relação alguma com suas chances de sobrevivência. Podemos então retirar essa coluna?<br>Bom, talvez. Vamos dar uma olhada em alguns dos nomes.

In [8]:
train['Name'].head()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

Os nomes em nossos conjuntos de dados são únicos, mas os títulos dos indivíduos (Mr., Mrs., Miss) se repetem. Vamos isolar os títulos de cada nome para analisarmos essa fato. A tabela abaixo mostra, para cada título, quantos homens e quantas mulheres o possuem.

O método *Series.str.extract()* extrai texto se baseando na regex utilizada.

In [9]:
for df in combine:
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


Antes de ver a relação dos títulos com sobrevivência, vamos agrupá-los. Talvez seja uma boa ideia deixar todos os mais raros em um único grupo, então vamos fazer isso.

In [10]:
for df in combine:
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                       'Don', 'Dr', 'Major', 'Rev', 'Sir', 
                                       'Jonkheer', 'Dona'], 'Rare')

    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


Muito interessante. O título de um passageiro realmente influencia sua chance de sobrevivência. Podemos manter essa feature que criamos e excluir a feature *Name*.

In [11]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

combine = [train, test]

### O passageiro está sozinho?

Uma pessoa que está sozinha tem mais chances de sobreviver? Abaixo criaremos uma feature que indica o tamanho da família do passageiro que está a bordo, e uma outra feature para indicar se um indivíduo está sozinho ou não.

In [12]:
for df in combine:
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = 0
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
    
train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


Aparentemente não. Você consegue imaginar um motivo para isso?

Vou excluir as features SibSp e Parch pois acho que as features que criamos já nos dão bastante informação. Você pode escolher mantê-las se preferir ou até mesmo testar posteriormente de que forma seu modelo foi melhor.

In [13]:
train = train.drop(['SibSp', 'Parch'], axis=1)
test = test.drop(['SibSp', 'Parch'], axis=1)

combine = [train, test]

### Como ficaram nossos datasets?

Vamos dar uma olhada nos nossos dois datasets (treino e teste) após a criação das features.

In [14]:
print(train.shape, test.shape)

(891, 12) (418, 11)


In [15]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Mr,2,0
1,2,1,1,female,38.0,PC 17599,71.2833,C85,C,Mrs,2,0
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,NaN,S,Miss,1,1
3,4,1,1,female,35.0,113803,53.1000,C123,S,Mrs,2,0
4,5,0,3,male,35.0,373450,8.0500,NaN,S,Mr,1,1


In [16]:
test.head()

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone
0,892,3,male,34.5,330911,7.8292,NaN,Q,Mr,1,1
1,893,3,female,47.0,363272,7.0000,NaN,S,Mrs,2,0
2,894,2,male,62.0,240276,9.6875,NaN,Q,Mr,1,1
3,895,3,male,27.0,315154,8.6625,NaN,S,Mr,1,1
4,896,3,female,22.0,3101298,12.2875,NaN,S,Mrs,3,0


Algumas dessas variáveis não nos dão muita informação, então podemos excluí-las. São elas *Ticket* e *Cabin*.

In [17]:
train = train.drop(['Ticket', 'Cabin'], axis=1)
test = test.drop(['Ticket', 'Cabin'], axis=1)

combine = [train, test]

In [18]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone
0,1,0,3,male,22.0,7.2500,S,Mr,2,0
1,2,1,1,female,38.0,71.2833,C,Mrs,2,0
2,3,1,3,female,26.0,7.9250,S,Miss,1,1
3,4,1,1,female,35.0,53.1000,S,Mrs,2,0
4,5,0,3,male,35.0,8.0500,S,Mr,1,1


## Feature encoding

O modelo que usaremos só aceita variáveis numéricas, por isso, precisaremos transformar nossos dados de alguma forma. Existem várias maneiras de fazer isso e nós vamos dar uma olhada em duas delas.

### Label encoding

Método que mapeia cada categoria em um número. É mais utilizado com métodos de árvore.

OBS: O For não é necessário aqui pois estamos encodando apenas uma variável, mas preferi deixar uma forma mais geral.

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
label_encoder_columns = ['Embarked']

for column in label_encoder_columns:
    # cria um encoder
    label_encoder = LabelEncoder()

    # cria um dicionario para o encoder
    label_encoder.fit(train[column])

    # aplica as transformações nos datasets
    train[column] = label_encoder.transform(train[column])
    test[column] = label_encoder.transform(test[column])

In [21]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone
0,1,0,3,male,22.0,7.2500,2,Mr,2,0
1,2,1,1,female,38.0,71.2833,0,Mrs,2,0
2,3,1,3,female,26.0,7.9250,2,Miss,1,1
3,4,1,1,female,35.0,53.1000,2,Mrs,2,0
4,5,0,3,male,35.0,8.0500,2,Mr,1,1


Como podemos ver, *Embarked* agora possui apenas valores numéricos. Caso seja necessário voltar aos valores iniciais em algum ponto do código, basta usar o método *LabelEncoder.inverse_transform()*. Abaixo temos um exemplo de como isso funciona.

In [22]:
label_encoder.inverse_transform(train['Embarked'])[0:5]

array(['S', 'C', 'S', 'S', 'S'], dtype=object)

### One-hot encoding

É um método que cria, para cada categoria, uma coluna com valores binários indicando a presença ou não da categoria na instância.

- Normalmente utilizado com métodos lineares, kNN e redes neurais.
- É necessário tomar cuidado caso existam muitos valores diferentes de categoria.

Evita que o modelo pense algo como: Mrs. > Miss > Mr. Deixa mais clara para o modelo a independência das categorias. Vamos aplicar em algumas colunas apenas para exemplificar. Para fazer isso podemos usar a função *pd.get_dummies()*

#### Exemplo de retorno da função

In [23]:
pd.get_dummies(train['Title'])

,Master,Miss,Mr,Mrs,Rare
0,0,0,1,0,0
1,0,0,0,1,0
2,0,1,0,0,0
3,0,0,0,1,0
4,0,0,1,0,0
...,...,...,...,...,...
886,0,0,0,0,1
887,0,1,0,0,0
888,0,1,0,0,0
889,0,0,1,0,0


#### Aplicação

In [24]:
one_hot_columns = ['Sex', 'Title']

one_hot = pd.get_dummies(train[one_hot_columns])
train = pd.concat([train, one_hot], axis=1).drop(one_hot_columns, axis=1)

one_hot = pd.get_dummies(test[one_hot_columns])
test = pd.concat([test, one_hot], axis=1).drop(one_hot_columns, axis=1)

In [25]:
train.head()

,PassengerId,Survived,Pclass,Age,Fare,Embarked,FamilySize,IsAlone,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,0,3,22.0,7.2500,2,2,0,0,1,0,0,1,0,0
1,2,1,1,38.0,71.2833,0,2,0,1,0,0,0,0,1,0
2,3,1,3,26.0,7.9250,2,1,1,1,0,0,1,0,0,0
3,4,1,1,35.0,53.1000,2,2,0,1,0,0,0,0,1,0
4,5,0,3,35.0,8.0500,2,1,1,0,1,0,0,1,0,0


## Feature scaling

Alguns algorítmos são sensíveis a escala dos dados e, por essa razão, precisamos colocar todos os dados na mesma escala para que esses modelos funcionem da melhor forma possível. Vamos dar uma olhada em dois dos métodos mais comuns para fazer isso.

É importante saber que uma Árvores de Decisão não é um desses modelos sensíveis, então só aplicaremos esses métodos aqui para exemplificação.

Como queremos aplicar essas tranformações nos nossos dados de treino e teste, é necessário que o Scaler tenha uma visão geral dos dados na hora de se adequar a eles, por isso, criaremos um dataset que concatena train e test.

In [26]:
df_concat = pd.concat([train.drop(['Survived'], axis=1), test], axis=0)

### Normalização

Faz com que todos os valores fiquem no intervalo [0, 1]:

$
\begin{align}
{x_{i}}' = \frac{x_{i} - min(x)}{max(x) - min(x)}
\end{align}
$

Podemos aplicar na coluna *Fare*.

In [27]:
from sklearn.preprocessing import MinMaxScaler

In [28]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(df_concat[['Fare']])

train[['Fare']] = scaler.transform(train[['Fare']])
test[['Fare']] = scaler.transform(test[['Fare']])

### Padronização

Faz com que os valores tenham média 0 e desvio padrão 1:

$
\begin{align}
{x_{i}}' = \frac{x_{i} -\mu}{\sigma}
\end{align}
$

Vamos utilizar a padronização na feature *Age* para exemplificar.

Explicação (em inglês, infelizmente) da importância de normalizar os padronizar os dados: https://humansofdata.atlan.com/2018/12/data-standardization/

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
scaler = StandardScaler()
scaler.fit(df_concat[['Age']])

train[['Age']] = scaler.transform(train[['Age']])
test[['Age']] = scaler.transform(test[['Age']])

## Resultado final

Vamos dar uma olhada final nos nossos dados antes se seguirmos em frente!

In [31]:
train.head()

,PassengerId,Survived,Pclass,Age,Fare,Embarked,FamilySize,IsAlone,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,0,3,-0.581628,0.014151,2,2,0,0,1,0,0,1,0,0
1,2,1,1,0.658652,0.139136,0,2,0,1,0,0,0,0,1,0
2,3,1,3,-0.271558,0.015469,2,1,1,1,0,0,1,0,0,0
3,4,1,1,0.426099,0.103644,2,2,0,1,0,0,0,0,1,0
4,5,0,3,0.426099,0.015713,2,1,1,0,1,0,0,1,0,0


In [32]:
test.head()

,PassengerId,Pclass,Age,Fare,Embarked,FamilySize,IsAlone,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,892,3,0.387341,0.015282,1,1,1,0,1,0,0,1,0,0
1,893,3,1.356310,0.013663,2,2,0,1,0,0,0,0,1,0
2,894,2,2.519072,0.018909,1,1,1,0,1,0,0,1,0,0
3,895,3,-0.194041,0.016908,2,1,1,0,1,0,0,1,0,0
4,896,3,-0.581628,0.023984,2,3,0,1,0,0,0,0,1,0


## Métricas de avaliação

Antes de começarmos a utilizar modelos para prever o futuro dos nossos passageiros, precisamos estudar um pouco sobre como avaliar o modelo que criarmos.

Métricas de avaliação são super importantes pois nos dão um valor que representa o quão bem nosso modelo está se saindo.

A métrica utilizada depende se estamos tratando de um problema de classificação ou regressão. Vamos ensinar algumas para cada um dos casos:

- Classificação
    - Acurácia
    - Precision e Recall
    - F-measure
    - AUC-ROC Curve
- Regressão
    - Erro quadrático médio (MSE)
    - R²

### Matriz de confusão

É uma medida de performance para modelos de classificação. É uma tabela com a combinação de valores previstos pelo modelo e valores reais:

<img src="https://miro.medium.com/max/712/1*Z54JgbS4DUwWSknhDCvNTQ.png" alt="confusion matrix">

Vamos entender o que cada uma das siglas representa fazendo uma analogia com um teste de alguma doença:

- TP (True Positive): Seu modelo previu positivo e estava correto. Esse caso seria equivalente a você **estar** doente e receber um **teste positivo**.

- TN (True Negative): Seu modelo previu negativo e estava correto. Esse caso é equivalente a você **não estar** doente e receber um **teste negativo**.

- FP (False Positive): Seu modelo previu positivo e estava incorreto: Esse caso é equivalente a você **não estar** doente e receber um **teste positivo**.

- FN (False Negative): Seu modelo previu negativo e estava incorreto. Esse caso é equivalente a você **estar** doente e receber um **teste negativo**.

Nossas previsões corretas ficam todas na diagonal principal da matrix. 

É importante perceber que a matrix de confusão pode ser usada para problemas de classficação de mais de 2 classes. Abaixo, temos um exemplo da matrix de confusão sendo usada em um problema de classificação de 3 classes:

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_001.png" alt="confusion matrix">



### Precision e Recall

**Precision**<br>
Qual a proporção de previsões positivas realmente corretas?

$
\begin{align}
Precision = \frac{TP}{TP+FP}
\end{align}
$

Um modelo sem falso positivo tem uma precisão de 1.0.

**Recall**<br>
Qual a proporção de observações realmente positivas foi identificada corretamente?

$
\begin{align}
Recall = \frac{TP}{TP+FN}
\end{align}
$

Um modelo sem falso negativo tem recall de 1.0.

### Acurácia

Indica a porcentagem de predições corretas que o modelo realizou.
Pode ser expressa da seguinte forma:

$
\begin{align}
Accuracy = \frac{TP+TN}{Total}
\end{align}
$

Pode enganar caso o número de ocorrências de cada classe esteja desbalanceado.

### F-measure

É uma medida de acurácia que leva em conta precision e recall ao mesmo tempo. Ela é dada por uma média harmônica das duas medidas:

$
\begin{align}
F_{1} = \frac{1}{recall^{-1} + precision^{-1}} = 2 \cdot \frac{recall \cdot precision}{recall+precision}
\end{align}
$

### AUC (Area under the curve)

Área abaixo de qual curva? Da curva **ROC** (Receiver operating characteristic).

E como funciona essa curva?

- Essa curva indica a relação entre as taxas de falsos positivos e verdadeiros positivos
- Nosso modelo terá como saída a probabilidade de classificar os dados como da classe 1
- Vamos supor que se P(Y=1) for > 0.5, dizemos que aquela instância é da classe 1, caso contrário classe 0.
- Isso irá nos dar uma taxa de falsos positivos e uma taxa de verdadeiros positivos e isso é um ponto na curva ROC.
- Agora escolhemos outros thresholds - diferentes de 0.5. Cada um desses gera um novo ponto na curva ROC.

<img src="https://miro.medium.com/max/751/1*RqK5DjVxcj4qZsCdN4FOSQ.png" alt="auc-roc">

Essa metrica não é sensível a distribuição das classes e, por essa razão, é comumente utilizada como métrica de diversas competições.

É importante notar que um **classificador aleatório** possui uma ROC-AUC de 0.5.

### Mean Squared Error

Método mais simples para avaliação de modelos de regressão. Basicamente, para cada instância, calculamos o quadrado da diferença entre o que deveríamos prever e o que previmos e então tiramos a média:

$
\begin{align}
MSE = \frac{1}{N} \sum_{i=0}^{N}(f(x_{i}) - y_{i})^{2}
\end{align}
$

### R² Score

Outro método para avaliar modelos de regressão. É também chamado de coeficiente de determinação e tem todos seus valores no intervalo [0,1].

$
\begin{align}
SStot = \sum_{i=0}(y_{i} - \bar{y}_{i})^{2}
\end{align}
$

$
\begin{align}
SSres = \sum_{i=0}(y_{i} - \hat{y}_{i})^{2}
\end{align}
$

$
\begin{align}
R^{2} = 1 - \frac{SSres}{SStot}
\end{align}
$

## Estratégias de validação

Com nossa métrica escolhida em mãos, chegou a hora de criar e validar nosso modelo!

Vamos aprender agora dois métodos de validação e, depois de validar nosso modelo, faremos uma submissão no Kaggle para ver o quanto nossa validação se aproxima do nosso resultado real no conjunto de teste.

Antes disso, vamos só preparar nossos dados para os processos.

In [33]:
train = train.drop(['PassengerId'], axis=1)

pass_ids = test['PassengerId']
test = test.drop(['PassengerId'], axis=1)

In [34]:
X = train.drop(['Survived'], axis=1)
y = train['Survived']

In [35]:
X.shape

(891, 13)

In [36]:
test.shape

(418, 13)

Perceba que agora temos o X e o test com as mesmas dimensões, isso vai facilitar as coisas para nós.

### Hold Out

Nessa estratégia, nós dividiremos nosso conjunto de dados em dois conjuntos. Usaremos um deles para treinar nosso modelo (conjunto de treino) e o outro para calcular a métrica de avaliação (conjunto de validação).

Nós sabemos a resposta correta para todo o nosso conjunto de dados mas vamos fingir que não sabemos para uma parte dele, usar nosso modelo pra prever essas respostas e depois comparar as predições com as respostas verdadeiras.

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [39]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

preds = model.predict(X_val)
print(f'Accuracy = {accuracy_score(y_val, preds)}')

# matriz de confusão do nosso modelo
confusion_matrix(y_val, preds)

Accuracy = 0.7932960893854749


array([[93, 19],
       [18, 49]])

### k-Fold Cross Validation

Nessa estratégia vamos dividir nossos dados em K conjuntos. Em cada iteração, utilizaremos K-1 desses conjuntos para treinamento e um deles para validação (variando qual conjunto é utilizado para validação em cada iteração). Depois disso, calculamos a média das iterações.

In [40]:
from sklearn.model_selection import KFold

In [41]:
NFOLDS = 5
folds = KFold(n_splits=NFOLDS)

columns = X.columns
score = 0

# criando os folds
splits = folds.split(X, y)

# para cada fold, pegaremos os index de treino e index de validação
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    model = DecisionTreeClassifier(max_depth=3)
    model.fit(X_train, y_train)
    
    y_pred_valid = model.predict(X_valid)
    print(f"Fold {fold_n + 1} | Accuracy: {accuracy_score(y_valid, y_pred_valid)}")
    
    score += accuracy_score(y_valid, y_pred_valid)
    
print(f"\nMean accuracy = {score/NFOLDS}")

Fold 1 | Accuracy: 0.8435754189944135
Fold 2 | Accuracy: 0.8089887640449438
Fold 3 | Accuracy: 0.8258426966292135
Fold 4 | Accuracy: 0.7696629213483146
Fold 5 | Accuracy: 0.8651685393258427

Mean accuracy = 0.8226476680685456


A ideia é utilizar a validação para escolher o melhor modelo e seus respectivos parâmetros.

## Prevendo para o teste e submetendo no Kaggle

Chegou a hora de usar o modelo que criamos e prever a resposta para os dados que não temos resposta!

In [42]:
# treinando o modelo com todos os dados
model = DecisionTreeClassifier(max_depth=3)
model.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [43]:
# usando o arquivo de exemplo para 
sub = pd.read_csv('gender_submission.csv')

In [44]:
test_preds = model.predict(test)
sub['Survived'] = test_preds

In [45]:
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [46]:
sub.to_csv('submission.csv', index=False)

Uma vez que você tem seu arquivo de predições, basta seguir o tutorial [nessa página](https://www.kaggle.com/c/titanic/overview) para aprender a submeter no site! E pronto!

### *Parabéns!!!*

Infelizmente, você vai perceber que nossa pontuação ainda não foi muito alta. Na verdade, ela foi só um pouco melhor que a submissão fornecida como exemplo. Mas tudo bem, nós ainda podemos melhorar muito! E isso nos leva ao nosso penúltimo tópico...

## É hora de ser criativo e botar a mão na massa!

Crie um novo notebook e faça as coisas do seu jeito! Esse notebook serviu apenas para te ensinar algumas técnicas, mas existe muito mais coisas para fazer. Crie novas features, processe os dados de forma diferente, teste e aprenda.

Uma boa fonte de aprendizado é a [sessão de Notebooks](https://www.kaggle.com/c/titanic/notebooks) dessa competição no Kaggle. Lá, competidores vão mostrar toda a sua solução e você pode aprender muito com eles. Não deixe de dar uma olhada.

## Considerações finais

Essa aula abordou diversos tópicos de pré-processamento dos dados e validação dos modelos, levando a uma solução, não ótima, mas completa do dataset do Titanic. É importante ter sempre em mente que cada tipo de modelo prefere os dados de uma certa forma quando realizando um pré-processamento. As técnicas ensinadas aqui podem te ajudar a processar os dados pra vários modelos.

Espero que essa aula tenha sido útil pra você. Não deixe de nos dar seu feedback, nós também estamos apredendo. Além disso, se você tiver qualquer dúvida, é só chamar no Telegram ou no Slack!

Obrigado pela atenção!